In [1]:
# -*- coding: utf-8 -*-

#  Licensed under the Apache License, Version 2.0 (the "License"); you may
#  not use this file except in compliance with the License. You may obtain
#  a copy of the License at
#
#       http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS, WITHOUT
#  WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the
#  License for the specific language governing permissions and limitations
#  under the License.

import os, sys, threading
import numpy as np
import wsgiref.simple_server as simple_server
import mysql.connector
import cv2
from mysql.connector import Error
from PIL import Image
from numpy import asarray
from time import time, gmtime, strftime, sleep
from argparse import ArgumentParser
from builtins import bytes
from linebot import LineBotApi, WebhookParser
from linebot.exceptions import InvalidSignatureError
from mos_package.mos_models import three_mosquitoes_three_models
from mos_package.database import mosquito_sql

from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageMessage, ImageSendMessage, LocationMessage
)
from linebot.utils import PY3


2022-09-14 10:47:24.308197: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Lirary is at /home/esidserver/Downloads/Zai/notebooks/mos_package


In [2]:
# get channel_secret and channel_access_token from your environment variable
channel_secret = 'c73718e5e705e7be5c3736c49c4863b5'
channel_access_token = 'MW/mgLewP4q1ko1rDC2hP4+RkKfnM3BPNK7yj0vT7sDwUJh2Wnb3ixSnHVwcpWgNTSpVuTRW5dUf9sXYhUg0MPdJX31PPMDiAsacCk+fy1bzQhUBkIsXe2TYKTMJdrpZM5QcD5iupo3Byjt+nyCEZgdB04t89/1O/w1cDnyilFU='
if channel_secret is None:
    print('Specify LINE_CHANNEL_SECRET as environment variable.')
    sys.exit(1)
if channel_access_token is None:
    print('Specify LINE_CHANNEL_ACCESS_TOKEN as environment variable.')
    sys.exit(1)

line_bot_api = LineBotApi(channel_access_token)
parser = WebhookParser(channel_secret)
mosDatabase = mosquito_sql()

In [3]:
# consumption = line_bot_api.get_message_quota_consumption()
# print(consumption)

In [4]:
database='mosquitoes'
file_path = './received.jpeg'
MosImagePre = './Mos'
JpegExtension = '.jpeg'

busyText = 'Thank you for submitting a mosquito photo, but someone is already using our system.  Please try again later.'
adviceText = 'Please submit only three jpeg photos of a mosquito subject, one-by-one; and share your location. Each photo should contain a mosquito image on a white or plain background.'
noMosText = 'No mosquito is found in your photo, retake once more.'
twoMosText = 'Two or more mosquitoes are found in your photo, retake once more.'
blurryText = 'This photo appears to be blurry, retake once more.'
firstReceivedText = 'First photo is well received.  Please send the second photo within 2 minutes.'
secondReceivedText = 'Second photo is well received.  Please send the last photo within 2 minutes.'
thirdReceivedText = 'Third and last photo is well received. Please share your location within 2 minutes.'
noLocationReceivedText = 'Send no more photos. Please share your location.'
locationReceivedText = 'Result: species of your mosquito is '


lock = lockOwner = imgIndex = timeOutFlag = 0
timeOutSec = 120 # 5minutes
images = np.zeros((3,512,512,3), dtype=np.uint8)
timeOutFlag = 0
ngrok = 'https://e5ff-161-200-85-86.ap.ngrok.io'

In [5]:
def SendTimeOutPushMessage(userID):
    line_bot_api.push_message(userID, TextSendMessage('Timeout!  Restart sending the first photo!'))
    return
    
def timeOutAction():
    global lock, lockOwner, imgIndex
    global timeOutFlag
    sendTimeOutPushMessage(lockOwner)
    lock = lockOwner = imgIndex = 0
    #timeOutInterrupt.cancel()
    #timeOutInterrupt = threading.Timer(timeOutSec,timeOutAction)
    timeOutFlag = 1
    return

def timeOutAction_test():
    global timeOutFlag
    global lock, lockOwner, imgIndex
    print('Timeout occurs!!!')
    timeOutFlag = 1
    lock = lockOwner = imgIndex = 0
    return

timeOutInterrupt = threading.Timer(timeOutSec, timeOutAction_test)

def restartTimer():
    global timeOutInterrupt
    timeOutInterrupt.cancel()
    timeOutInterrupt = threading.Timer(timeOutSec, timeOutAction_test)
    timeOutInterrupt.start()
    return

def timeOutAcknowledge():
    timeOutFlag = 0
    return

def YOLOv5(img):
    noImgFound = 1
    image = Image.open(file_path)
    image = image.resize((512,512))
    data = asarray(image)    
    return noImgFound, data

    
def Ensemble(images):
    species = 0
    confidence = 0.95
    return species, confidence

def blurry(im_rgb, threshold = 170):
    #note expect rgb numpy array as input - > bgr (cv2) -> gray
    im_bgr = cv2.cvtColor(im_rgb, cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(im_bgr, cv2.COLOR_BGR2GRAY)
    # find laplacian variance 
    fm = cv2.Laplacian(gray, cv2.CV_64F).var()
    # if less than theshold -> blurry
    FilterResult = (fm < threshold)
    return FilterResult

def create_body(text):
    if PY3:
        return [bytes(text, 'utf-8')]
    else:
        return text



In [6]:
def application(environ, start_response):
    global imgIndex, lock, lockOwner, images
    global timeOutInterrupt
    # check request path
    path_info = environ['PATH_INFO']
    if path_info != '/callback':
        # check if it is /images
        if path_info.lower().find('images') != 1:
            start_response('404 Not Found', [])
            return create_body('Not Found')
        else:
            headers = [('Content-type', 'image/jpeg')]
            start_response('200 OK', headers)
            data = b''
            filename = file_path
            with open(filename, 'rb', buffering=0) as f:
                data = f.readall()
            #print(type(data))       #<class 'bytes'>
            return [data]   
        
    else:
        # check request method
        if environ['REQUEST_METHOD'] != 'POST':
            start_response('405 Method Not Allowed', [])
            return create_body('Method Not Allowed')
        # get X-Line-Signature header value
        signature = environ['HTTP_X_LINE_SIGNATURE']

        # get request body as text
        wsgi_input = environ['wsgi.input']
        content_length = int(environ['CONTENT_LENGTH'])
        body = wsgi_input.read(content_length).decode('utf-8')

        # parse webhook body
        try:
            events = parser.parse(body, signature)
        except InvalidSignatureError:
            start_response('400 Bad Request', [])
            return create_body('Bad Request')

        
        for event in events:
            # check if event is a message event, if not try next (continue) if any 
            if not isinstance(event, MessageEvent):
                continue
            # if a message event, check if image message or location message, if not give comments and return    
            if not isinstance(event.message, ImageMessage) and not isinstance(event.message, LocationMessage):
                errorMessage = [TextSendMessage(adviceText), ImageSendMessage(
                  original_content_url = ngrok + '/images/mos.jpg',
                  preview_image_url = ngrok + '/images/mos.jpg')
                ]
                line_bot_api.reply_message(event.reply_token, errorMessage)
                if lock == 1:
                    restartTimer()
                start_response('200 OK', [])
                return create_body('OK') 
            # if an image message and valid image index         
            if isinstance(event.message, ImageMessage) and imgIndex < 3:
                # reach here when first, second or third image is being received.
                # check if lock or not, if so lock by user ID or not   
                if lock !=0 and lockOwner != event.source.user_id:
                    # tell the user to try again later
                    errorMessage = TextSendMessage(busyText)
                    line_bot_api.reply_message( event.reply_token, errorMessage)
                    start_response('200 OK', [])
                    return create_body('OK')

                #read the photo    
                message_content = line_bot_api.get_message_content(event.message.id)
                with open(file_path, 'wb') as fd:
                    for chunk in message_content.iter_content():
                        fd.write(chunk)
                    fd.close()
                # send file to YOLO, which return the number of mosquito, and the numpy image of the first mos.     
                noMos, img = YOLOv5(file_path)
                # Note img should be center-cropped and have 512x512 pixels. 
                if noMos == 0:
                    errorMessage = TextSendMessage(noMosText)
                    line_bot_api.reply_message(event.reply_token, errorMessage)
                    start_response('200 OK', [])
                    return create_body('OK')
                elif noMos > 1:
                    errorMessage = TextSendMessage(twoMosText)
                    line_bot_api.reply_message(event.reply_token, errorMessage)
                    start_response('200 OK', [])
                    return create_body('OK')
                # check if the image is blurry 
                if blurry(img) == True:
                    errorMessage = TextSendMessage(BlurryText)
                    line_bot_api.reply_message(event.reply_token, errorMessage)
                    start_response('200 OK', [])
                    return create_body('OK')
                # preprocessing ends save the image
                # check if unlock? if so lock and restart the process, start timer    
                if lock == 0:
                    lock = 1
                    imgIndex = 0
                    lockOwner = event.source.user_id
                # start counting Timeout    
                restartTimer()
                images[imgIndex] = img
                imgIndex += 1
                if imgIndex == 1:
                    goodMessage = TextSendMessage(firstReceivedText)
                    line_bot_api.reply_message(event.reply_token, goodMessage)
                    start_response('200 OK', [])
                    return create_body('OK')
                elif imgIndex == 2:
                    goodMessage = TextSendMessage(secondReceivedText)
                    line_bot_api.reply_message(event.reply_token, goodMessage)
                    start_response('200 OK', [])
                    return create_body('OK')
                elif imgIndex == 3:
                    goodMessage = TextSendMessage(thirdReceivedText)
                    line_bot_api.reply_message(event.reply_token, goodMessage)
                    start_response('200 OK', [])
                    return create_body('OK')
            elif isinstance(event.message, ImageMessage) and imgIndex == 3:
                # reach here when too many images had been sent
                errorMessage = TextSendMessage(noLocationReceivedText)
                line_bot_api.reply_message(event.reply_token, errorMessage)
                restartTimer()
                start_response('200 OK', [])
                return create_body('OK')
            elif isinstance(event.message, LocationMessage) and imgIndex ==3: #LocationMessage
                # reach here when all the right steps had been met.
                # prepare location
                print('received three images and location!')
                location_title = event.message.title
                location_address = event.message.address
                latitude = event.message.latitude
                longitude = event.message.longitude
                # prepare file to write from 3 images
                file1 = 'tmp1.jpg'
                file2 = 'tmp2.jpg'
                file3 = 'tmp3.jpg'

                im_nd = images[0]
                im = Image.fromarray(im_nd)
                im = im.save(file1)

                im_nd = images[1]
                im = Image.fromarray(im_nd)
                im = im.save(file2)
                
                im_nd = images[2]
                im = Image.fromarray(im_nd)
                im = im.save(file3)
                # perform prediction send the three images to 
                threeImages = three_mosquitoes_three_models(images[0], images[1], images[2])
                species_name, confidence = threeImages.ensemble_fake()
                resultMessage = TextSendMessage(locationReceivedText + species_name + '.')
                # save all data to database
                idx = int(time())
                struct_time = (gmtime(idx+7*60))
                dt = strftime("%Y-%m-%d %H:%M:%S", struct_time)

                mosDatabase.writeThreeMosquitoImgLocationPredictionToDatabase(idx, file1, file2, file3, 
                                                      location_title, location_address, latitude, longitude, dt,
                                                      species_name, confidence)
                line_bot_api.reply_message(event.reply_token, resultMessage)
                timeOutInterrupt.cancel()
                lock = lockOwner = imgIndex = 0
                start_response('200 OK', [])
                return create_body('OK')
            else:
                # reach here when at least 1 photo had been received but didn't follow right sequence
                errorMessage = TextSendMessage(adviceText + ' '+ str(imgIndex) + ' photo(s) received!')
                line_bot_api.reply_message(event.reply_token, errorMessage)
                restartTimer()
                start_response('200 OK', [])
                return create_body('OK')
                
                


In [ ]:
if __name__ == '__main__':
#     arg_parser = ArgumentParser(
#         usage='Usage: python ' + __file__ + ' [--port <port>] [--help]'
#     )
#     arg_parser.add_argument('-p', '--port', type=int, default=5000, help='port')
#     options = arg_parser.parse_args()

    path = 'public' 
    port = 5000
    httpd = simple_server.make_server('', port, application)
    print("Serving {} on port {}, control-C to stop".format(path, port))
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        print("Shutting down.")
        httpd.server_close()

Serving public on port 5000, control-C to stop
